<hr>

<p>
<font size=10px color="navy"><center><b>Income Qualification Project
<P>


<hr>   
    
<font size = 5px  color ='red'><center><b>Identify the level of income qualification needed for the families in Latin America.</b></font>
    
<hr>
    
    
    
<font size = 5px><b>Problem Statement Scenario:</b> </font>
    
    
    
    
Many social programs have a hard time making sure the right people are given enough aid. It’s tricky when a program focuses on the poorest segment of the population. This segment of population can’t provide the necessary income and expense records to prove that they qualify.

In Latin America, a popular method called Proxy Means Test (PMT) uses an algorithm to verify income qualification. With PMT, agencies use a model that considers a family’s observable household attributes like the material of their walls and ceiling or the assets found in their homes to classify them and predict their level of need. While this is an improvement, accuracy remains a problem as the region’s population grows and poverty declines.

The Inter-American Development Bank (IDB) believes that new methods beyond traditional econometrics, based on a dataset of Costa Rican household characteristics, might help improve PMT’s performance.


   
<hr>


<font color = "#191970" size=6px><center><b>Import Required Libraries</b></font>

In [ ]:
# basic
import pandas as pd, numpy as np

# Viusalization 
import seaborn as sns
from matplotlib import pyplot as plt, style

# Feature Engineering/ Feature Selection
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler



# Model Bulding and performance metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

# Set warnings off
import warnings
warnings.filterwarnings('ignore')

<font color = "#191970" size=6px><center><b>Data Collection

In [ ]:
# Training Data 

df_train = pd.read_csv("/kaggle/input/costa-rican-household-poverty-prediction/train.csv")

# Testing Data

df_test = pd.read_csv("/kaggle/input/costa-rican-household-poverty-prediction/test.csv")

<font color = "#191970" size=8px><center><b>Data Cleaning And EDA

In [ ]:
# Viewing The Shape of test amd traim data

print("\n")
print("Shape of Train data : ",df_train.shape)
print("Shape of Test data : ",df_test.shape)
print("\n")

##       1. Identify the output variable.

In [ ]:
# Test Data Doesnt Have Output column therefore we check the missing column in test data

print("\n")
for column in df_train.columns:
    if column not in df_test.columns:
        print("Output Variable : ",column)
print("\n")

## 2. Understand the type of data.

In [ ]:
# Info of data

df_train.info()

In [ ]:
# Getting List Of Data types For each Columns 

print("\n")
print('Integer Columns :', list(df_train.select_dtypes(np.int64).columns))
print("\n")
print('Float Columns :', list(df_train.select_dtypes(np.float64).columns))
print("\n")
print('Categorical Columns :', list(df_train.select_dtypes(np.object).columns))
print("\n")


In [ ]:
# Count Of each Datatype cols

print("\n")
print(" Count for each datatype columns : \n\n",df_train.dtypes.value_counts())
print("\n")

In [ ]:
# Exploring categorical Features

print('\n')
for column in df_train.columns:
    type = df_train[column].dtype
    if type == 'object':
        print(column)

print('\n')

<hr>

<font color = "red" size=4px><b> Data Dictionary For Above Object Variables </font>

- Id : Unique ID
- idhogar :   Household level identifier
- dependency :   Dependency rate, calculated = (number of members of the household younger than 19 or older than 64)/(number of member of household between 19 and 64)
- edjefe :   Years of education of Male head of household, based on the interaction of escolari (years of education), head of household and gender, yes=1 and no=0
- edjefa :   Years of education of Female head of household, based on the interaction of escolari (years of education), head of household and gender, yes=1 and no=0
    
<font color = "blue" size=4px> <b>Interpretation :</b>*</font><font color = "red" size=4px> As Id and idhogar will have no significant variance for prediction,  we will drop them</font>
<hr>

In [ ]:
# Print Categorical Features

print('\n')
for column in df_train.columns:
    type = df_train[column].dtype
    if type == 'object':
        print(column)

print('\n')

In [ ]:
# Checking For Unique Values of each Categorical Features

print("\n")
print("Dependency : \n",df_train.dependency.value_counts())
print("\n")
print("edjefe : \n",df_train.edjefe.value_counts())
print("\n")
print("edjefe :\n ",df_train.edjefe.value_counts())
print("\n")


In [ ]:
# Function for encoding categorical values in column

def encode_to_numeric(value):
    if value == 'yes':
        return(float(1))
    elif value == 'no':
        return(float(0))
    else:
        return(float(value))
    
# Applying Fucntion to Categorical Features in train data

cols=['dependency','edjefe','edjefa']

for col in cols:
    df_train[col] = df_train[col].apply(encode_to_numeric)

# Applying Fucntion to Categorical Features in test data

for col in cols:
    df_test[col] = df_test[col].apply(encode_to_numeric)

In [ ]:
# Check if any categorical Features Remaining..

print("\n")
print(" Count for each datatype columns : \n\n",df_train.dtypes.value_counts())
print("\n")

## 3. Count how many null values are existing in columns.

In [ ]:
# Check for missing Values:

# Missing Values for integer features
print("\n")
null_counts_int = df_train.select_dtypes(np.int64).isnull().sum()
print("Integer Variables : ",null_counts_int[null_counts_int > 0])
print("\n")

# Missing Values for Float features

null_counts_float = df_train.select_dtypes(np.float64).isnull().sum()
print("Float Variables :\n\n ",null_counts_float[null_counts_float > 0])
print("\n")

# Missing Values in Target Variable

print('Target variable : ', df_train.Target.isna().sum())
print("\n")

<hr>

<font color = "red" size=4px><b> Data Dictionary For Above Float Variables </font>

- v2a1 : Monthly rent payment 
- v18q1 : number of tablets household owns
- rez_esc : Years behind in school

- SQBmeaned : square of the mean years of education of adults (>=18) in the household
- meaneduc : average years of education for adults (18+)
    

<hr>

<font color = "maroon" size=4px> Lets Check the missing rent payments with respect to house ownership status

In [ ]:
# Variables Indication House Ownership Status: 

ownership_status_cols = [column for column in df_train if column.startswith('tipovivi')]

# Plot Missing Rents count for each Ownership status

style.use('ggplot')
df_train.loc[df_train.v2a1.isnull(), ownership_status_cols].sum().plot.bar(figsize = (10, 8), color = 'royalblue', edgecolor = 'k', linewidth = 3)
plt.xticks(ticks=[0, 1, 2, 3, 4],labels = ['Owns and Paid Off', 'Owns and Paying', 'Rented', 'Precarious', 'Other'])
plt.title('Missing Rents count for each Ownership status', size = 15,pad=15);                                                              

<hr>

<font color = "blue" size=4px> <b>Interpretation :</b></font><font color = "red" size=4px> From above Intepretation we can infere that house which is owned and fully paid has most missing rent and which makes sense, so we will fill all the missing rents with 0</font>
   
<hr>

In [ ]:
# Filling missing rents with zero for train and test data

df_train.v2a1.fillna(0,inplace=True)
df_test.v2a1.fillna(0,inplace=True)

<font color = "maroon" size=4px>Lets Check the missing number of tablets household owns with respect to (v18q) if head of house (parentesco1) owns a tablet or not

In [ ]:
# rows for head of house

head_of_house = df_train.loc[df_train.parentesco1 == 1].copy()

# Group the number of talets a household owns with head of house owing a tablet or not

Count_na_talets = pd.DataFrame(head_of_house.groupby('v18q')['v18q1'].apply(lambda x: x.isnull().sum())).reset_index().rename(columns={'v18q1':'missing Value Count for number of tablest household owns','v18q':'Head of house Owning Tablet'})

# Plotting The count of Missing tablets count of household with respect to Ownership of tablet of Head of House

plt.figure(figsize = (12,6))
sns.barplot(Count_na_talets['Head of house Owning Tablet'],Count_na_talets['missing Value Count for number of tablest household owns'],color='cyan')
plt.title('The count of Missing tablets count of household with respect to Ownership of tablet of Head of House')
plt.show()

<hr>

<font color = "blue" size=4px> <b>Interpretation :<b></font> <font color = "red" size=4px> From above Intepretation we can infere that if Head of House doesnt owns a tablet, there will be zero number of tablets a household owns , so we will replace all missing count value with 0 </font>
<hr>

In [ ]:
# Filling missing tablets value with zero in train amd test data sets

for column in [df_train,df_test]:
    column['v18q1'].fillna(0,inplace=True)

<font color = "maroon" size=4px> Lets Look at rez_esc : Years behind in school with null values(7928),
    columns related is Age lets compare it 



In [ ]:
df_train[df_train['rez_esc'].notnull()]['age'].describe()

<font color = "maroon" size=3px> min age is 7 and max is 17 


In [ ]:
# rez_esc has some value when except the nulls lets Verify it

df_train.loc[df_train['rez_esc'].isnull()]['age'].describe()

<font color = "maroon" size=3px> the column has value when not null , thus we will not drop it

In [ ]:
# lets find the age which is behind school in range of AGE 7-17

df_train.loc[df_train.rez_esc.isna() & (df_train.age > 7) & (df_train.age < 17)]['age']

<hr>

<font color = "blue" size=4px> <b>Interpretation :</b></font><font color = "red" size=4px> From above Intepretation we can infere that there's only one member in family who's is behind in school with age of 10, lets fill it with zero</font>
<hr>

In [ ]:
# Filling null values with Zero in Train and test data

for column in [df_train,df_test]:
    column['rez_esc'].fillna(0,inplace=True)

<hr>

Lets look at meaneduc   (total nulls: 5) : average years of education for adults (18+)  
 why the null values, Lets look at few rows with nulls in meaneduc


Columns related to average years of education for adults (18+)  
- edjefe
- edjefa
- instlevel1, =1 no level of education
- instlevel2, =1 incomplete primary 

In [ ]:
# Lets eplore few rows of meaneduc null values

meaneduc_null_rows = df_train[df_train.meaneduc.isnull()].head()

columns = ['edjefe','edjefa','instlevel1','instlevel2']

meaneduc_null_rows[columns][meaneduc_null_rows[columns]['instlevel1'] > 0].count()

<hr>

<font color = "blue" size=4px> <b>Interpretation :</b></font><font color = "red" size=4px> From above Intepretation we can infere that meaneduc is null when no level of education is 0 , lets fill the missing values with na for both meaneduc and SQBmeaned</font>
<hr>

In [ ]:
# Filling The SQBmeaned amd meaneduc with 0 values in train and test data

for column in [df_train,df_test]:
    column['meaneduc'].fillna(0,inplace = True)
    column['SQBmeaned'].fillna(0,inplace = True)

In [ ]:
# lets check for Missing Values 
df_train.isnull().sum().value_counts()

## 4. Check if there is a house without a family head.

In [ ]:
# Grouping Household with head of house counts

house_wihtout_heads = pd.DataFrame(df_train.groupby("idhogar")['parentesco1'].sum().value_counts().reset_index()).rename(columns={'index':'Heads(Yes/no)','parentesco1':'Household counts'})

house_wihtout_heads

<hr>

- parentescol1 : 1 if household head


<font color = "blue" size=4px> <b>Interpretation :</b></font><font color = "red" size=4px> From above Intepretation we can infere that there are <b>15</b> Households Without Family head and <b>2973</b> Households with Family Head</font>
<hr>

## 5. Check whether all members of the house have the same poverty level.

In [ ]:
# Group the housholds with target values and get unique Counts

all_equal = df_train.groupby("idhogar")['Target'].apply(lambda x : x.nunique()==1)

# Get number of Households Which are not having same target value

not_equal = all_equal[all_equal != True]

print("\n")
print("Number of households with family members not having similar poverty level : ",len(not_equal))
print("Number of households with family members having similar poverty level : ",len(all_equal))
print("\n")

<hr>

<font color = "blue" size=4px> <b>Interpretation :</b></font> <font color = "red" size=4px> From above Intepretation we can infere that there are <b>85*</b>Households with family members not having similar poverty level and <b>2988</b> with  similar poverty level</font>
<hr>

## 6. Set the poverty level of the members and the head of the house same in a family.

In [ ]:
# Iterate through each household


for household in not_equal.index:
    
    # Find the correct label (for the head of household)
    
    true_target = int(df_train[(df_train['idhogar'] == household) & (df_train['parentesco1'] == 1.0)]['Target'])
    
    # Set the correct label for all members in the household
    
    df_train.loc[df_train['idhogar'] == household, 'Target'] = true_target
    
    
# Groupby the household and figure out the number of unique values

all_equal = df_train.groupby('idhogar')['Target'].apply(lambda x: x.nunique() == 1)

# Households where targets are not all equal

not_equal = all_equal[all_equal != True]
print("\n")
print('There are {} households where the family members do not all have the same target.'.format(len(not_equal)))
print("\n")

## 7. Check if there are any biases in our dataset.

<hr>

<font color = "red" size=4px> Target(Poverty Level):-</font>

- 1 = extreme poverty 
- 2 = moderate poverty
- 3 = vulnerable households
- 4 = non vulnerable households 

In [ ]:
# Getting value_counts for each target

target_counts = df_train.Target.value_counts().sort_index()

# Plot The Count for each Poverty Level
print("\n")
target_counts.plot.bar(figsize = (12,6),linewidth=3,edgecolor='k')
plt.xlabel("Poverty Levels",size = 16)
plt.ylabel("Count",size = 16)
plt.xticks([0,1,2,3],['extreme poverty','moderate poverty','vulnerable households','non vulnerable households'],rotation=20)
plt.title("Poverty Level Counts",pad=30,size=16)
plt.show()

<hr>

<font color = "blue" size=4px> <b>Interpretation :</b></font><font color = "red" size=4px> From above Intepretation we can infere that  Extreme Poverty has samallest count whereas Non Vulnerable households are having highest count , Therefore the <b>Data is Biased<b></font>
<hr>

## 8. Clean the data and remove variables which are not significant 

In [ ]:
# Remove Unwanted columns from train and test

columns = ['SQBescolari', 'SQBage', 'SQBhogar_total', 'SQBedjefe','SQBhogar_nin', 'SQBovercrowding', 'SQBdependency', 'SQBmeaned', 'agesq','Id','idhogar']

for df in [df_train,df_test]:
    df.drop(columns=columns,inplace=True)

#### 8.1. Remove Features with Zero Variance

In [ ]:
# Check for Columns with Zero Variance In data Set

var_thres = VarianceThreshold(threshold=0.001)

features_no_output = df_train.drop(columns='Target')
var_thres.fit(features_no_output)

# getting columns with Zero variance

constant_columns = [column for column in features_no_output.columns if column not in features_no_output.columns[var_thres.get_support()]]

print("Varible with Zero Variance : ",constant_columns)

<hr>

<font color = "blue" size=4px> <b>Interpretation :</b></font><font color = "red" size=4px> From above Intepretation we can infere that </font>

- elimbasu5 : (1 if rubbish disposal mainly by throwing in river, creek or sea). is having zero varaince 
- pisoother : 1 if predominant material on the floor is other
- planpri  : 1 electricity from private plant
    
 <font color = "red" size=4px> are having almost zero Variance ,so we will drop it<>/font
    
<hr>

In [ ]:
# Dropping columns with zeri variance in Train and test data

for column in [df_train,df_test]:
    column.drop(constant_columns,axis=1,inplace=True)

#### 5.2. Lets Remove Features which are Highly Correlated

In [ ]:
# Creating a function through which we can select highly correlated features and dropping the first feature that is correlated with any other feature

def correlation(dataset, threshold):
    col_corr = set() # Set of all names of correlated columns
    corr_matrix = dataset.corr() # Correlation Matrix
    
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold:    # we are interested in absolute coeff value
                    
                colname = corr_matrix.columns[i] # getting the name of columns
                col_corr.add(colname)
    return(col_corr)


In [ ]:
# Getting the Features with High Correlation

features_no_output = df_train.drop(columns=['Target'])

corr_features = correlation(features_no_output,0.90) # Setting Threshold as features having correlation above 85%
print("\n")
print("Correlated Features :\n ",corr_features)
print("\n")
print("No. of Features Correlated: ",len(corr_features))

<hr>

<font color = "blue" size=4px> <b>Interpretation :</b></font><font color = "red" size=4px> From above Intepretation we can infere that Using Pearson Correlation we were able find out 10 Highly Correlated Features with <b>90% Correlation</b> with other Features In DataSet</font>

<font color = "red" size=4px> We can remove those features to avoid dimentionality curse</font>

<hr>

In [ ]:
# Dropping the Highly Correlated Features From Train and Test Data set

for df in [df_train,df_test]:
    df.drop(columns=corr_features,inplace = True)



In [ ]:
# Check if any correlated features Remaining

corr_features = correlation(df_train,0.90) # Setting Threshold as features having correlation above 85%
print("\n")
print("No. of Features Correlated: ",len(corr_features))
print("\n")

<font color = "#191970" size=8px><center><b> Model Building

#### 1. Defining Features and Target

In [ ]:
Features = df_train.drop(columns='Target')
Target = df_train.Target

#### 1.1 define Train Test Split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(Features,Target,random_state = 1)

#### 2. Standardise the data

In [ ]:
Features_std = StandardScaler().fit_transform(Features)
x_train_std = StandardScaler().fit_transform(x_train)
x_test_std = StandardScaler().fit_transform(x_test)

In [ ]:
# Shape Of Split
print("Shape Of Train : ",x_train_std.shape)
print("Shape Of Test  : ",x_test_std.shape)

#### 3. Model Training

In [ ]:
# Instantiate estimator Object

rfmodel1 = RandomForestClassifier(criterion='entropy', max_depth=25, min_samples_leaf=1,
                       min_samples_split=2, n_estimators=5000,n_jobs=-1,random_state=1)

# Fit The Model

rfmodel1.fit(x_train_std,y_train)

# Performance Metrics
print("\n")
print("Train Score : ",rfmodel1.score(x_train_std,y_train))
print("Test  Score : ",rfmodel1.score(x_test_std,y_test))

##### 

In [ ]:
# KFold Cross Validation

kf = KFold(n_splits=10,shuffle=True,random_state=1)

In [ ]:
# Instantiate Estimator Object 

rfModel2 = RandomForestClassifier()

# Fit the Model

rfModel2.fit(Features_std,Target)

#Predictions

prediction = model_selection.cross_val_predict(rfModel2,Features_std,Target,cv=kf)



In [ ]:
print("Classification Report : \n\n",metrics.classification_report(Target,prediction))

In [ ]:
pd.crosstab(Target,prediction)

<hr>

<font color = "blue" size=4px> <b>Interpretation :</b></font><font color = "red" size=4px> Model Achieved <b>95% Accuracy</b> cosidering all Features</font>

<font color = "red" size=4px> Lets now take Most Significant Features</font>

<hr>

### Get Important Features

In [ ]:
# Get Important Features

labels = list(x_train)
feature_importances = pd.DataFrame({'Importance':rfmodel1.feature_importances_},index=labels).sort_values(by='Importance',ascending=False)

In [ ]:
# Features Having almost zero variance

feature_importances[feature_importances.Importance < 0.0001]

In [ ]:
# lets plot them

feature_importances.Importance.sort_values(ascending=True).plot(kind='barh', figsize=(20, 20))
plt.xlabel('Importance')

plt.title("Feature Importances")
plt.show()


In [ ]:
print("Number Of Features : ", feature_importances.value_counts().sum())

<hr>

<font color = "blue" size=4px> <b>Interpretation :</b></font><font color = "red" size=4px> From above Figure we can infere that Most features are having almost <b>zero significance</b> to the model</font>

<font color = "red" size=4px> Lets now take Most Significant Features with <b>Threshold Of 0.01</b></font>

<hr>

In [ ]:
# Taking most Significant Features

feature_importances = feature_importances[feature_importances.Importance > 0.01]

feature_importances.Importance.sort_values(ascending=True).plot(kind='barh', figsize=(15, 11),color='royalblue')
plt.xlabel('Importance')
plt.title("Feature Importances")
plt.show()

print("\n")

In [ ]:
print("Number Of Features : ", feature_importances.value_counts().sum())

<hr>

<font color = "blue" size=4px> <b>Interpretation :</b></font><font color = "red" size=4px> From above Figure we can infere that <b>meaneduc and dependency</b> are having highest significance to the model</font>

<font color = "red" size=4px> We have reduced Features From <b>118 to 32</b></font>

<hr>

<font color = "#191970" size=8px><center><b> Model Building With Significant Variables

In [ ]:
# Getting Most Significant Features names

Sig_features = feature_importances.index

#### 1. Defining Features and Target

In [ ]:
Features = df_train[Sig_features]
Target = df_train.Target

#### 1.1 define Train Test Split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(Features,Target,random_state = 1)

#### 2. Standardise the data

In [ ]:
Features_std = StandardScaler().fit_transform(Features)
x_train_std = StandardScaler().fit_transform(x_train)
x_test_std = StandardScaler().fit_transform(x_test)

In [ ]:
# Shape Of Split
print("Shape Of Train : ",x_train_std.shape)
print("Shape Of Test  : ",x_test_std.shape)

#### 3. Grid Search

In [ ]:
# Grid Search 
grid_rf2= GridSearchCV(RandomForestClassifier(),param_grid={'n_estimators':[100],'criterion':['entropy'],
                                                             'max_depth':[30,35],'min_samples_split':[1,2,3],
                                                             'min_samples_leaf':[1,2,3]},n_jobs=-1)

grid_rf2.fit(x_train_std,y_train)

In [ ]:
# Getting Best Parameters 
grid_rf2.best_params_

#### 3. Model Training

In [ ]:
# Instantiate estimator Object

rfmodel2 = RandomForestClassifier(criterion='entropy', max_depth=31, min_samples_leaf=1,
                       min_samples_split=2, n_estimators=1000,n_jobs=-1,random_state=1)

# Fit The Model

rfmodel2.fit(x_train_std,y_train)

# Performance Metrics
print("\n")
print("Train Score : ",rfmodel2.score(x_train_std,y_train))
print("Test  Score : ",rfmodel2.score(x_test_std,y_test))
print("\n")

##### Cross Validation

In [ ]:
# KFold Cross Validation

kf = KFold(n_splits=10,shuffle=True,random_state=1)

In [ ]:
# Instantiate Estimator Object 

rfModel2_sig = RandomForestClassifier(criterion='entropy', max_depth=31, min_samples_leaf=1,
                       min_samples_split=2, n_estimators=1000,n_jobs=-1,random_state=1)

# Fit the Model

rfModel2_sig.fit(Features_std,Target)

#Predictions

prediction = model_selection.cross_val_predict(rfModel2_sig,Features_std,Target,cv=kf)



In [ ]:
print("Classification Report : \n\n",metrics.classification_report(Target,prediction))

In [ ]:
pd.crosstab(Target,prediction)

<hr>

<font color = "blue" size=4px> <b>Interpretation :</b></font>

<font color = "red" size=4px> Model Achieved <b>97% Accuracy</b> cosidering Significant Features</font>

<font color = "red" size=4px> We increased Accuracy From <b>95% to 97%</b> below are Classification report:</font>
    
<font color = "blue" size=4px> <b>CLassification Report Model With All Features :</b></font> 
    
  
               precision    recall  f1-score   

           1       0.99      0.88      0.93   
           2       0.98      0.90      0.94   
           3       0.99      0.83      0.90   
           4       0.93      1.00      0.97   

    accuracy                           0.95   


    
    
<font color = "blue" size=4px> <b>CLassification Report Model With Significant :</b></font>  
    
    
               precision    recall  f1-score   

           1       0.98      0.93      0.96   
           2       0.98      0.94      0.96   
           3       0.99      0.91      0.95    
           4       0.96      1.00      0.98      

    accuracy                           0.97   
 
 

    
    
    
<hr>

## 8. Now lets do prediction on Unseen Test Data

#### 1. Select Most significant Features For test Data Set

In [ ]:
df_test_sig = df_test[Sig_features]

df_test_sig.shape

#### 2. Standardise the data

In [ ]:
df_test_sig = StandardScaler().fit_transform(df_test_sig)

#### 3. Prediction

In [ ]:
#Predictions

Prediction = pd.Series(rfModel2_sig.predict(df_test_sig))

#### 4. Plot The Count for each Predicted Poverty Level

In [ ]:
predicted_counts = Prediction.value_counts().sort_index()

# Plot The Count for each Predicted Poverty Level
print("\n")
predicted_counts.plot.bar(figsize = (12,6),linewidth=3,edgecolor='k',color='royalblue')
plt.xlabel("Poverty Levels",size = 16)
plt.ylabel("Count",size = 16)
plt.xticks([0,1,2,3],['extreme poverty','moderate poverty','vulnerable households','non vulnerable households'],rotation=20)
plt.title("Poverty Level Counts (Predicted)",pad=30,size=16)
plt.show()

In [ ]:
# Lets Read test data to get id coloum

test_df_submission =  pd.read_csv("/kaggle/input/costa-rican-household-poverty-prediction/test.csv")

In [ ]:
test_df_submission.columns

In [ ]:
# Save Prediction in Csv File
my_submission = pd.DataFrame({'Id': test_df_submission.Id, 'Target': Prediction})
# you could use any filename. We choose submission here
my_submission.to_csv('submission.csv', index=False)
